In [133]:
# -*- coding: utf-8 -*-
import numpy as np
import urllib.request, json, re
import pprint as pp
import tensorflow as tf
import re

from tensorflow.python.framework import ops
import matplotlib.pyplot as plt
import time
import datetime


import random
np.random.seed(1)

# Klassformat för lagring och serialisering
    

In [134]:
class DocList:
    data = []
    def __init__(self):
        self.name='MotionList'
        self.data = []
    def reprJSON(self):
        return dict(name=self.name, data=self.data) 
    
class Motion:
    def __init__(self):
        self.text=''
        self.title=''
        self.subtype=''
        self.meta = {}
    def reprJSON(self):
        return dict(text=self.text, subtype=self.subtype, title=self.title, meta=self.meta) 
    
class DocSerializer(json.JSONEncoder):
    def default(self, obj):
        if hasattr(obj,'reprJSON'):
            return obj.reprJSON()
        else:
            return json.JSONEncoder.default(self, obj)



## Import och skapande av datafil
denna koden ska bara köras en gång vid inläsning av data från riksdagen

In [136]:

def import_web_data(datasize=3):

    #http://data.riksdagen.se/dokumentlista/?doktyp=mot
    #http://data.riksdagen.se/dokument/H502225.text (exempeldok.)
    #"http://data.riksdagen.se/anforandelista/?rm=&anftyp=Nej&d=&ts=&parti=s&iid=0218878014918&sz="+str(datasize)+"&utformat=json"
    #http://data.riksdagen.se/anforandelista/?rm=&anftyp=&d=&ts=&parti=&iid=&sz=10&utformat=xml
    motioner = DocList()
    with urllib.request.urlopen("http://data.riksdagen.se/dokumentlista/?doktyp=mot&sz="+str(datasize)+"&utformat=json") as url:
        rawdata = json.loads(url.read().decode())
        motionslista = rawdata['dokumentlista']['dokument']
        for meta in motionslista:
            if(not not meta):
                motion = Motion()
                with urllib.request.urlopen('http:' + meta['dokument_url_text']) as text:
                    motion.text = re.sub('\n','',text.read().decode())
                    #motion.meta = meta (save space appr 30%)
                    motion.title = meta['titel']
                    motion.subtype = meta['subtyp']

                motioner.data.append(motion)

     
    toJson = json.dumps(motioner.reprJSON(), cls=DocSerializer,  sort_keys=True, indent=4)
    with open('motioner.txt','w',encoding='utf-8') as f:
        
        f.write(toJson) #json.dumps(motions.reprJSON()))#json.dumps(motions, sort_keys=True, indent=4))
        
    return toJson, motioner

#datasize är antal dokument vi behöver förmodligen ett hundratal iaf. (nedan är bortkommenterad om man har fil)
#data_ex, motionList = import_web_data(datasize=500)
print("totalt antal dokument: " + str(len(motionList.data)))



totalt antal dokument: 200


In [137]:
def file_to_json(filename):
    with open(filename,encoding="utf-8") as f:
        data = json.loads(f.read())
        return data
    
def json_to_vec(jsondata):
    docs = []
    words = []
    for sample in jsondata:
        #remove non letters
        cleaned = sample["text"].lower()
        cleaned = re.sub('[\r\r]', '', cleaned) #måste vara såhär (då ord kan vara delade varsomhelst med \r\r)
        cleaned = re.sub('[\n|\)|\(]', ' ', cleaned)
        PERMITTED_CHARS = "abcdefghijklmnopqrstuvwxyzåäöABCDEFGHIJKLMNOPQRSTUVWXYZÅÄÖ "

        cleaned = "".join(c for c in cleaned if c in PERMITTED_CHARS)
        #cleaned = re.sub(r'^\w', '', sample["text"].lower())
        #cleaned =  sample["text"].lower()
        #remove space
        cleaned =' '.join( [x for x in re.split(r'[\u200b\s]+', cleaned, flags=re.UNICODE) if x != ''])
        words += cleaned.lower().split()
        docs.append(cleaned)

    #print(speech)
    #data = tf.compat.as_str(speech).split()
    orig_dic = dict(list(zip(words,words)))
    return docs, orig_dic

In [139]:
jsondata = file_to_json("motioner.txt")

docs, orig_dic = json_to_vec(jsondata['data'])

lista = list(set(orig_dic.keys()))
lista.sort()
print(lista[6300:6310])

['hemavdrag', 'hembesök', 'hembygd', 'hembygden', 'hemelektronik', 'hemifrån', 'hemkunskap', 'hemkunskapsämnet', 'hemland', 'hemlandet']


#  läs från fil

In [140]:
jsondata = file_to_json("motioner.txt")

docs, orig_dic = json_to_vec(jsondata['data'])

print(docs[3])


motion till riksdagenav oscar sjöstedt och dennis dioukarev båda sd med anledning av prop bättre förutsättningar för fondsparande och hållbara valförslag till riksdagsbeslutriksdagen avslår förslaget i de delar som avser krav på att tillhandahållande av hållbarhetsinformation regleras i lag och därtill knutna bestämmelser i förslagen till lag om värdepappersfonder och lag om förvaltare av alternativa investeringsfonder avsnitt och motiveringregeringens föreslår att genom lag reglera att fonder ska lämna information om förvaltning med avseende på olika hållbarhetsaspekter så som miljö sociala förhållanden personal respekt för mänskliga rättigheter och korruption vidare ska samtliga fonder vare sig de beaktar hållbarhetsaspekter eller ej också åläggas med informationskrav om att rapportera detta det finns idag flera internationellt erkända alternativ för hållbarhetsinformation fn initiativet principles for responsible investments pri som förtydligar redovisning inom hållbarhet med kapita

#  Lemmatisering av alla ord
Här måste vi gå igenom alla ord och rensa upp för att sedan ersätta ord i texten med en mindre ordrymd

In [141]:
def indexOf(text, elem):
        try:
            thing_index = text.index(elem)
            return thing_index
        except ValueError:
            return -1

        
#skapar en dictionary över gammalt till nytt ord så man kan köra find-replace med den senare.
def lemmatize_and_clean(orig_dic, sensitivity=.5):        
    # a jph production yeahh
    
    # unika ord (set)
    myset = set(list(orig_dic.keys()))
    # till lista igen
    all_words = list(myset)
    #sortera bokstav
    all_words.sort(reverse=False)
    
 
    orig = all_words
    newlist = []
    #regexp = re.compile('(.?[0-9]+)|^[/&%\[=\–]') #rensa tecken
    for i in range(len(orig)):
        #if(regexp.match(orig[i])):
        #    newlist.append('')
        #else:
            fidx = 0
            if(len(newlist)>0):
                suffix=0
                if(len(orig[i])>7):
                    suffix = 2
                fidx = (indexOf(orig[i],newlist[i-1][:len(orig[i])-suffix])+1)
            #kolla om längd på aktuellt ersättningsord (newlist[i-1]) är över dellängd av nuvarande ord (eg .6 = 60%)
            #ifall denna skillnad blir för stor byt ersättningsord
            if(fidx != 0 and ((1.0*len(newlist[i-1])/len(orig[i]) >=sensitivity) and len(newlist[i-1]) > 4)):
                newlist.append(newlist[i-1])
            else:
                newlist.append(orig[i])
    
    
    return dict(list(zip(orig,newlist)))


orig_dic  = lemmatize_and_clean(orig_dic,sensitivity=.4)

print("from "+str(len(orig_dic))+" to: "+str(len(set(list(orig_dic.values())))))

#print(orig_dic)


from 19479 to: 11476


TODO: använd wdict på alla textdokument för att ersätta affärens, affärsmodell, affärsrunda... med affär (enl ovan)

# Stop words
ta bort ord som inte säger något som prepositioner mm. Min naiva approach är att kolla upp de vanligaste orden och ta bort dem. Tror dock man ska kolla (http://stevenloria.com/finding-important-words-in-a-document-using-tf-idf/)
St (pip install stop-words yay)

In [142]:
from stop_words import get_stop_words

def remove_stop_words(dic):
    stop_words = get_stop_words('sv')
    return {k:('' if (dic[k] in stop_words) else dic[k]) for k in dic}
    

In [143]:
jsondata = file_to_json("motioner.txt")
texts, orig_dic = json_to_vec(jsondata['data'])

stop_dic = remove_stop_words(orig_dic)
print("remove Stopwords from "+str(len(orig_dic))+" to: "+str(len(set(list(stop_dic.values())))))

lemma_dic = lemmatize_and_clean(stop_dic,sensitivity=.4)
print("remove suffix from "+str(len(set(list(stop_dic.values()))))+" to: "+str(len(set(list(lemma_dic.values())))))

remove Stopwords from 19479 to: 19373
remove suffix from 19373 to: 11476


# Preprocessing

In [144]:
def build_dict(orig_dic):
    before = str(len(orig_dic.keys()))
    dic = remove_stop_words(orig_dic)
    dic = lemmatize_and_clean(dic,sensitivity=.4)
    print("Limit words from "+before+" to: "+str(len(set(list(dic.values()))))+"\n")
    return dic

def preprocess(texts,dic):
    lst = [re.escape(key) for key in dic.keys()]
    pattern = re.compile(r'\b(' + '|'.join(lst) + r')\b')
    cleaned = []
    for text in texts:
        cleaned.append(pattern.sub(lambda x: dic[x.group()], text))
    return cleaned

In [145]:
def batch_clean(file_in, file_out):
    jsondata = file_to_json(file_in)
    texts, orig_dic = json_to_vec(jsondata['data'])
    dic = build_dict(orig_dic)
    texts = preprocess(texts,dic)
    with open(file_out,'w',encoding='utf-8') as f:
        f.write(json.dumps(texts, sort_keys=True, indent=4))
        

In [146]:
batch_clean('motioner.txt','motioner_cleaned.txt')

Limit words from 19479 to: 11476



## Test

In [147]:
#ta ut alla dokument
jsondata = file_to_json("motioner.txt")
texts, orig_dic = json_to_vec(jsondata['data'])

dic = build_dict(orig_dic)
#print(dic)
ts = texts[2:3]
print(ts,len(ts[0]))
texts = preprocess(ts,dic)

print("\n",texts,len(texts[0]))


Limit words from 19479 to: 11476

['motion till riksdagenav olle felten och david lång båda sd med anledning av prop mervärdesskatt vid omsättning som avser fartyg samt deklarationstidpunkt i vissa fallförslag till riksdagsbeslutriksdagen ställer sig bakom det som anförs i motionen om att även sjöfart som går mellan två stater där territorialvattengränserna tangerar varandra bör omfattas och detta tillkännager riksdagen för regeringenmotiveringi propositionen föreslås bland annat enändring i mervärdesskattelagen i fråga omundantag från skatteplikt för omsättning av vissa fartyg samt förnödenheter till dessa innebörden av förslaget är att undantaget inskränks tillsådana fartyg som används på vad man definierar som öppna havet och som fraktar passagerare mot betalning eller används för handels industri eller fiskeriverksamhet samt fartyg som används för sjöräddningassistans eller kustfiskesjöfart som går mellan två stater där territorialvattengränserna tangerar varandra skulle därmed om 

# Nästa steg - Bag of words
skapa vektor av alla ord. Eg bara köra list(set(orig_dic.values())) 
Skapa np.arrays där varje dokument har en siffra i varje position i ovan vektor.
Detta blir då ett training example eller ett x i X.

# NMF

för att sedan få matrisrepresentation som går att köra NonNegative Matrix Factorization på.

http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.NMF.html